# Tratamento dos dados
*Objetivos*: 
- Tratar o arquivo NINDINET.DBF para que seja possível a análise dos dados.
- Gerar um arquivo painel.parquet que será consumido pelo Streamlit

In [1]:
import os
import pandas as pd
from dbfread import DBF
from filtros import agravo, ano, faixa_etaria, raca, sexo


In [2]:
colunas_df = ['NU_NOTIFIC', 'TP_NOT', 'ID_AGRAVO', 'CS_SUSPEIT','IN_AIDS', 'CS_MENING',
           'DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'SG_UF_NOT', 'ID_MUNICIP', 'ID_REGIONA',
           'ID_UNIDADE', 'DT_SIN_PRI', 'SEM_PRI', 'DT_NASC', 'NU_IDADE_N', 'CS_SEXO', 
           'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N', 'ID_CNS_SUS', 'SG_UF', 'ID_MN_RESI', 
           'ID_RG_RESI', 'ID_DISTRIT', 'ID_BAIRRO', 'NM_BAIRRO', 'NU_CEP', 'NU_DDD_TEL', 
           'NU_TELEFON', 'CS_ZONA', 'ID_PAIS', 'NDUPLIC_N', 'IN_VINCULA', 'DT_INVEST', 
           'ID_OCUPA_N', 'CLASSI_FIN', 'CRITERIO', 'TPAUTOCTO', 'COUFINF', 'COPAISINF',
           'COMUNINF', 'CODISINF', 'CO_BAINFC', 'NOBAIINF', 'DOENCA_TRA', 'EVOLUCAO',
           'DT_OBITO', 'DT_ENCERRA', 'DT_DIGITA', 'DT_TRANSUS', 'DT_TRANSDM', 'DT_TRANSSM',
           'DT_TRANSRM', 'DT_TRANSRS', 'DT_TRANSSE', 'NU_LOTE_V', 'NU_LOTE_H', 'CS_FLXRET', 
           'FLXRECEBI', 'TPUNINOT']

colunas_painel = ['ID_AGRAVO', 'NU_ANO', 'ID_MN_RESI', 'CS_SEXO', 'CS_RACA', 'CS_ESCOL_N', 'DT_OBITO', 'NU_IDADE_N']

## Convertendo o arquivo .dbf
- Converter o arquivo .xls para .csv

In [3]:
# Converter de .dbf para .csv
# dbf = DBF('dbf/NINDINET.DBF')
# df = pd.DataFrame(iter(dbf))
# df.to_csv('NINDINET.csv', index=False)
# 
# df.head()

- CASO JÁ TENHA CRIADO O ARQUIVO .CSV, PODE EXECUTAR À PARTIR DAQUI
- Descomentar o código abaixo e comentar o código acima

In [4]:
# Carrega o .csv salvo
arquivo_csv = 'NINDINET.csv'

df = pd.read_csv(arquivo_csv, usecols=colunas_painel, sep=',', low_memory=False)

df.head()

,ID_AGRAVO,NU_ANO,NU_IDADE_N,CS_SEXO,CS_RACA,CS_ESCOL_N,ID_MN_RESI,DT_OBITO
0,A90,2007,4029,M,4.0,2.0,150304.0,NaN
1,A90,2007,4018,F,9.0,9.0,150506.0,NaN
2,A90,2008,4002,M,4.0,10.0,150330.0,NaN
3,A90,2008,4025,F,4.0,6.0,150553.0,NaN
4,X29,2008,4047,M,4.0,9.0,150450.0,NaN


## Funções Auxiliares

In [5]:
# Função para retornar os valores únicos das colunas do df
def valores_unicos(df):
    for coluna in df.columns:
        print(f'{coluna}: {df[coluna].unique()}')
        print()

# valores_unicos(df[colunas_painel])

In [6]:
# Função para verificar a existência de valores nulos nas colunas do df
def verifica_nulos(df):
    for coluna in df.columns:
        print(f'{coluna}: {df[coluna].isnull().sum()}')
        print()


In [7]:
# Função para converter a idade
def converter_idade(idade):
    faixas_etarias = {
        (0, 999): 'IGNORADO',
        (1000, 3999): '00 a 01 ano',
        (4000, 4004): '01 a 04 anos',
        (4005, 4009): '05 a 09 anos',
        (4010, 4014): '10 a 14 anos',
        (4015, 4019): '15 a 19 anos',
        (4020, 4029): '20 a 29 anos',
        (4030, 4039): '30 a 39 anos',
        (4040, 4049): '40 a 49 anos',
        (4050, 4059): '50 a 59 anos',
        (4060, 4069): '60 a 69 anos',
        (4070, 4079): '70 a 79 anos',
        (4080, 4999): 'Mais de 80 anos'
    }

    for (inicio, fim), faixa in faixas_etarias.items():
        if inicio <= idade <= fim:
            return faixa
    return 'IGNORADO'

## Tratamento de valores nulos

In [8]:
verifica_nulos(df[colunas_painel])

ID_AGRAVO: 0

NU_ANO: 0

ID_MN_RESI: 61

CS_SEXO: 0

CS_RACA: 51617

CS_ESCOL_N: 196246

DT_OBITO: 1654827

NU_IDADE_N: 0



In [9]:
# Substituir os valores nulos do df por 9
df = df.fillna(9)

# Verifica se sobrou mais algum valor nulo
verifica_nulos(df[colunas_painel])

ID_AGRAVO: 0

NU_ANO: 0

ID_MN_RESI: 0

CS_SEXO: 0

CS_RACA: 0

CS_ESCOL_N: 0

DT_OBITO: 0

NU_IDADE_N: 0



## Recodicar os valores das colunas
- Usar os dicionários de recodificação para substituir os valores das colunas
- Os valores foram obtidos no dicionário de dados e estão disponíveis na pasta filtros

In [10]:
# Tratando a coluna ID_AGRAVO, substituindo os códigos pelos nomes dos agravos no dicionário AGRAVO
df['ID_AGRAVO'] = df['ID_AGRAVO'].map(agravo.AGRAVO)

# Tratando a coluna CS_SEXO, substituindo os códigos pelos nomes dos sexos no dicionário sexo.SEXO
df['CS_SEXO'] = df['CS_SEXO'].map(sexo.SEXO)

# Tratando a coluna CS_RACA, substituindo os códigos pelos nomes das raças no dicionário raca.RACA
df['CS_RACA'] = df['CS_RACA'].map(raca.RACA)

# Tratando a coluna NU_IDADE_N, aplicando a função converter_idade
df['NU_IDADE_N'] = df['NU_IDADE_N'].apply(converter_idade)

df.head()

,ID_AGRAVO,NU_ANO,NU_IDADE_N,CS_SEXO,CS_RACA,CS_ESCOL_N,ID_MN_RESI,DT_OBITO
0,A90 - DENGUE,2007,20 a 29 anos,Masculino,Parda,2.0,150304.0,9
1,A90 - DENGUE,2007,15 a 19 anos,Feminino,Ignorado,9.0,150506.0,9
2,A90 - DENGUE,2008,01 a 04 anos,Masculino,Parda,10.0,150330.0,9
3,A90 - DENGUE,2008,20 a 29 anos,Feminino,Parda,6.0,150553.0,9
4,X29 - ACIDENTE POR ANIMAIS PECONHENTOS,2008,40 a 49 anos,Masculino,Parda,9.0,150450.0,9


In [11]:
# Convertendo a coluna ID_MN_RESI para inteiro
df['ID_MN_RESI'] = df['ID_MN_RESI'].astype(int)

# Convertendo a coluna CS_ESCOL_N para inteiro
df['CS_ESCOL_N'] = df['CS_ESCOL_N'].astype(int)

# Removendo as / da coluna DT_OBITO
df['DT_OBITO'] = df['DT_OBITO'].str.replace('/', '')

verifica_nulos(df[colunas_painel])

ID_AGRAVO: 227

NU_ANO: 0

ID_MN_RESI: 0

CS_SEXO: 0

CS_RACA: 0

CS_ESCOL_N: 0

DT_OBITO: 1654827

NU_IDADE_N: 0



In [12]:
df_regiao = pd.read_csv('filtros/dim_regiao.csv', sep=';')
df_regiao.head()


,mun_cod,mun_nome,macro_reg_saude,reg_saude,centro_reg_saude,reg_integracao
0,150010,Abaetetuba,Macro I,Tocantins,06 CRS,Tocantins
1,150013,Abel Figueiredo,Macro IV,Carajas,11 CRS,Capim
2,150020,Acara,Macro II,Metropolitana II,02 CRS,Tocantins
3,150030,Afua,Macro I,Marajo I,07 CRS,Marajo
4,150034,Agua Azul do Norte,Macro IV,Araguaia,12 CRS,Araguaia


In [13]:
# Renomear as colunas de código dos municípios para permitir a junção
df.rename(columns={'ID_MN_RESI': 'mun_cod'}, inplace=True)

# Unir os dois dataframes com base na coluna de código dos municípios
df_merged = pd.merge(df, df_regiao, on='mun_cod', how='left')

# Exibir o dataframe unido para o usuário
df_merged.head()

,ID_AGRAVO,NU_ANO,NU_IDADE_N,CS_SEXO,CS_RACA,CS_ESCOL_N,mun_cod,DT_OBITO,mun_nome,macro_reg_saude,reg_saude,centro_reg_saude,reg_integracao
0,A90 - DENGUE,2007,20 a 29 anos,Masculino,Parda,2,150304,NaN,Floresta do Araguaia,Macro IV,Araguaia,12 CRS,Araguaia
1,A90 - DENGUE,2007,15 a 19 anos,Feminino,Ignorado,9,150506,NaN,Novo Repartimento,Macro IV,Lago de Tucurui,11 CRS,Lago de Tucurui
2,A90 - DENGUE,2008,01 a 04 anos,Masculino,Parda,10,150330,NaN,Igarape-Miri,Macro I,Tocantins,06 CRS,Tocantins
3,A90 - DENGUE,2008,20 a 29 anos,Feminino,Parda,6,150553,NaN,Parauapebas,Macro IV,Carajas,11 CRS,Carajas
4,X29 - ACIDENTE POR ANIMAIS PECONHENTOS,2008,40 a 49 anos,Masculino,Parda,9,150450,NaN,Melgaco,Macro I,Marajo II,08 CRS,Marajo


In [14]:
df_merged.isnull().sum()

ID_AGRAVO               227
NU_ANO                    0
NU_IDADE_N                0
CS_SEXO                   0
CS_RACA                   0
CS_ESCOL_N                0
mun_cod                   0
DT_OBITO            1654827
mun_nome               3833
macro_reg_saude        3833
reg_saude              3833
centro_reg_saude       3833
reg_integracao         3833
dtype: int64

In [15]:
df_merged = df_merged.fillna('IGNORADO')

In [16]:
df_merged.rename(columns={
    'mun_cod': 'MUN_COD',
    'mun_nome': 'MUN_NOME',
    'macro_reg_saude': 'MACRO_REG_SAUDE',
    'reg_saude': 'REG_SAUDE',
    'centro_reg_saude': 'CENTRO_REG_SAUDE',
    'reg_integracao': 'REG_INTEGRACAO',
}, inplace=True)

df_merged.head()

,ID_AGRAVO,NU_ANO,NU_IDADE_N,CS_SEXO,CS_RACA,CS_ESCOL_N,MUN_COD,DT_OBITO,MUN_NOME,MACRO_REG_SAUDE,REG_SAUDE,CENTRO_REG_SAUDE,REG_INTEGRACAO
0,A90 - DENGUE,2007,20 a 29 anos,Masculino,Parda,2,150304,IGNORADO,Floresta do Araguaia,Macro IV,Araguaia,12 CRS,Araguaia
1,A90 - DENGUE,2007,15 a 19 anos,Feminino,Ignorado,9,150506,IGNORADO,Novo Repartimento,Macro IV,Lago de Tucurui,11 CRS,Lago de Tucurui
2,A90 - DENGUE,2008,01 a 04 anos,Masculino,Parda,10,150330,IGNORADO,Igarape-Miri,Macro I,Tocantins,06 CRS,Tocantins
3,A90 - DENGUE,2008,20 a 29 anos,Feminino,Parda,6,150553,IGNORADO,Parauapebas,Macro IV,Carajas,11 CRS,Carajas
4,X29 - ACIDENTE POR ANIMAIS PECONHENTOS,2008,40 a 49 anos,Masculino,Parda,9,150450,IGNORADO,Melgaco,Macro I,Marajo II,08 CRS,Marajo


In [17]:
valores_unicos(df_merged)


ID_AGRAVO: ['A90 - DENGUE' 'X29 - ACIDENTE POR ANIMAIS PECONHENTOS'
 'W64 - ATENDIMENTO ANTI-RABICO'
 'B551 - LEISHMANIOSE TEGUMENTAR AMERICANA'
 'N72 - SINDROME DO CORRIMENTO CERVICAL EM MULHERES'
 'A630 - CONDILOMA ACUMINADO (VERRUGAS ANOGENITAIS)'
 'B550 - LEISHMANIOSE VISCERAL' 'B24 - AIDS' 'B19 - HEPATITES VIRAIS'
 'A60 - HERPES GENITAL' 'A53 - SIFILIS EM ADULTO'
 'R17 - SINDROME ICTERICA AGUDA' 'A539 - SIFILIS NAO ESPECIFICADA'
 'Y09 - VIOLENCIA INTERPESSOAL - AUTOPROVOCADA'
 'B09 - DOENCAS EXANTEMATICAS' 'O981 - SIFILIS EM GESTANTE'
 'Y96 - ACIDENTE DE TRABALHO GRAVE' 'A010 - FEBRE TIFOIDE'
 'A379 - COQUELUCHE' 'A959 - FEBRE AMARELA' 'B659 - ESQUISTOSSOMOSE'
 'Z209 - ACIDENTE DE TRABALHO COM EXPOSICAO A MATERIAL BIOLOGICO'
 'A509 - SIFILIS CONGENITA' 'A309 - HANSNENIASE'
 'B571 - DOENCA DE CHAGAS AGUDA' 'G039 - MENINGITE'
 'N485 - SINDROME DA ULCERA GENITAL (EXCLUIDO HERPES GENITAL)'
 'R36 - SINDROME DO CORRIMENTO URETRAL EM HOMEM' 'A279 - LEPTOSPIROSE'
 'B019 - VARICELA SEM COM

In [18]:
# Salvando o arquivo tratado
#df_merged.to_csv('painel.csv', index=False)

df_merged.to_parquet('painel.parquet', index=False)